# Facial Expression Recognition - Using FER2013 Database and Transfer Learning
**Author**: Christopher Holzweber

**Matr.-Nr.**: k11803108

**Description**: Bachelorthesis - Prototype for FER


In [ ]:
# %tensorflow_version 2.x  # making sure using version 2 of tensorflow
import tensorflow as tf  # import tensorflow module
import numpy as np # standard lib. for calculations
import cv2  # library for imagehandling 
import matplotlib.pyplot as plt  # plotting images
import os  # for file handling and loading
import pandas as pd
import random # for datashuffling
from sklearn.metrics import confusion_matrix # for evaluationg the CNN Model
import seaborn as sn # pi install seaborn - used for plotting confusion matrix

# Dataset
For this project the dataset FER2013 is used

https://www.kaggle.com/msambare/fer2013

The data is already seperated into a test and a train dataset.

The Fer2013 classifies facial emotions into 7 Categories:

(0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral)

If using Google Colab, upload zip-File into content folder and unzip the train and test images

**Example Image - Angry **

In [ ]:
# !unzip /content/FER2013 #unzip needed when importing data zip file in google colab

In [ ]:
img = cv2.imread("./FER2013/train/0/Training_3908.jpg") # random angry image (index 0)

In [ ]:
plt.imshow(img)  # show angry image

*Define Classes and Data/Label Arrays*

In [ ]:
emotion_classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
train_data = [] # picture data for model training
train_label = [] # labels of training data
test_data = [] # picture data for model testing
test_label = []  # labels of testing data
IMG_SIZE = 100 #set pixel size of image 
imagetype = 1 # 0 for grayscale, 1 for rgb

**Function: Read all  Images from subfolders and return data  + label tuples**

In [ ]:
def readData(direction,nrsubfolders):
    dataframe = []
    traindir = direction
    for cs in range(0, nrsubfolders):
        path = os.path.join(traindir,str(cs))  # Iterate over every subfolder
        for img in os.listdir(path):
            tempImg = cv2.imread(os.path.join(path,img),imagetype)  # load image 0 for greyscale
            tempImg = cv2.resize(tempImg, (IMG_SIZE, IMG_SIZE))
            dataframe.append([tempImg,cs])
    return dataframe

**get all training and test images and shuffle them**

In [ ]:
tempdata = readData("./FER2013/train/",len(emotion_classes)) # read training data
random.shuffle(tempdata) #shuffle data
#store data in according arrays
for feat,label in tempdata:
    train_data.append(feat)
    train_label.append(label)
tempdata = readData("./FER2013/test/",len(emotion_classes)) # read testing data
random.shuffle(tempdata) #shuffle data
#store data in according arrays
for feat,label in tempdata:
    test_data.append(feat)
    test_label.append(label)

# create arrays out of lists
train_data = np.array(train_data)
train_label = np.array(train_label)
test_data = np.array(test_data)
test_label = np.array(test_label)



In [ ]:
IMG_INDEX = 100
plt.imshow(train_data[IMG_INDEX])
plt.xlabel(train_label[IMG_INDEX])

In [ ]:
train_data.shape

In [ ]:
train_label.shape

In [ ]:
test_data.shape

In [ ]:
# normalize datavalues for machinelearning - best practice that values are between [0, 1]
train_data, test_data = train_data / 255.0, test_data / 255.0

# SetUp CNN Architecture

In [ ]:
from tensorflow.keras import layers, models  # use models and layers given by tensorflow framework

In [ ]:
# Model Skeleton reused form the Paper Facial Emotion Recognition Using Deep Convolutional Neural Network
model2020 = models.Sequential()
model2020.add(layers.Conv2D(64, (3,3), activation='relu', input_shape=(IMG_SIZE,IMG_SIZE, 3)))
model2020.add(layers.MaxPooling2D((2, 2)))
model2020.add(layers.Dropout(.01))
model2020.add(layers.Conv2D(32, (3,3), activation='relu'))
model2020.add(layers.MaxPooling2D((2, 2)))
model2020.add(layers.Dropout(.01))
model2020.add(layers.Conv2D(32, (3, 3), activation='relu'))
model2020.add(layers.MaxPooling2D((2, 2)))
model2020.add(layers.Dropout(.01))
model2020.add(layers.Flatten())
model2020.add(layers.Dense(256, activation='relu'))
model2020.add(layers.Dense(7, activation = 'softmax'))
model2020.summary()

# *Training Area*

In [ ]:
model2020.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model2020.fit(train_data, train_label, epochs=15, 
                    validation_data=(test_data, test_label))

In [ ]:
model2020.save('./SavedModels/model2020_fer2013_p100.h5')  # Save Model in Modeldirectory

# *Testing and Validation Area*

In [ ]:
acc = history.history['accuracy']

In [ ]:
loaded_model = tf.keras.models.load_model("./SavedModels/model2020_fer2013_p100.h5")

In [ ]:
test_loss, test_acc = model.evaluate(test_data, test_label, verbose=2)

Test with Happy Boy Image

In [ ]:
# test with happy boy
img = cv2.imread("./manualFaces/happyboy.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img =  cv2.resize(img, (48, 48))
img = np.array(img)
plt.imshow(img)
img = img / 255.0
temp = [] # reshape testdata, becuase predict function needs 4 dimensions
temp.append(img)
temp = np.array(temp)
temp.shape
predictions = loaded_model.predict(temp)
emotion_classes[np.argmax(predictions)]

In [ ]:
## Create Confusion Matrix of learn model

In [ ]:
def getConfusionMatrix():
    predicted = model2020.predict(test_data)
    y_pred = []
    for i in range(0, 7178):
        y_pred.append(np.argmax(predicted[i,:]))
    y_pred = np.array(y_pred)
    return confusion_matrix(test_label, y_pred)

In [ ]:
def printConfusionMatrix()
    conmatrix = getConfusionMatrix()
    df_cm = pd.DataFrame(conmatrix, index = [i for i in "0123456"],
                  columns = [i for i in "0123456"])

    plt.figure(figsize = (10,7))
    sn.heatmap(df_cm, annot=True,fmt="d")
    plt.title('Confusion Matrix of CNN Model')
    # Set x-axis label
    plt.xlabel('Predicted')
    # Set y-axis label
    plt.ylabel('Actual')

In [ ]:
printConfusionMatrix()